# 1. Setup and Load Data

## 1.1 Install Dependencies and Setup

In [4]:
%pip install tensorflow opencv-python matplotlib

  Using cached tensorflow-2.13.0rc2-cp311-cp311-macosx_12_0_arm64.whl (2.0 kB)
  Using cached opencv_python-4.8.0.74-cp37-abi3-macosx_11_0_arm64.whl (33.1 MB)
  Using cached matplotlib-3.7.1-cp311-cp311-macosx_11_0_arm64.whl (7.3 MB)
  Using cached tensorflow_macos-2.13.0rc2-cp311-cp311-macosx_12_0_arm64.whl (189.3 MB)
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl (26 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached h5py-3.9.0-cp311-cp311-macosx_11_0_arm64.whl (2.6 MB)
  Using cached libclang-16.0.0-py2.py3-none-macosx_11_0_arm64.whl (24.3 MB)
  Using cached numpy-1.25.0-cp311-cp311-macosx_11_0_arm64.whl (14.0 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached protobuf-4.23.3-cp37-abi3-macosx_10_9_universal2.whl (400 kB)
  Using cached termcolor-2.3.0-py3-no

## 1.2 Remove Dodgy Images

## 1.3 Load Data